<a href="https://colab.research.google.com/github/erickcori/RL-ETAR-BSM1/blob/main/notebooks/03_agente_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Entrenamiento de Agente RL
# Este notebook prepara y entrena un agente de aprendizaje por refuerzo (PPO/DQN) usando la simulación del BSM1.

In [4]:
# Entrenamiento de Agente RL
# Este notebook prepara y entrena un agente de aprendizaje por refuerzo (PPO) usando la simulación del BSM1 (modo demostración).

# Instalar librerías necesarias
!pip install stable-baselines3[extra] optuna -q



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 24.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.9/231.9 kB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
import gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from gym import spaces

# Simulación simplificada del BSM1 (mock)
class BSM1Env(gym.Env):
    def __init__(self):
        super(BSM1Env, self).__init__()
        self.action_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)  # Control de oxígeno
        self.observation_space = spaces.Box(low=0, high=100, shape=(4,), dtype=np.float32)  # DQO, NH4, NO3, SRT
        self.state = np.array([50.0, 10.0, 5.0, 8.0], dtype=np.float32)
        self.t = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = np.array([50.0, 10.0, 5.0, 8.0], dtype=np.float32)
        self.t = 0
        return self.state, {}

    def step(self, action):
        oxigeno = action[0]
        self.state += np.random.normal(0, 0.5, size=self.state.shape) - oxigeno
        reward = -np.sum(np.square(self.state - np.array([30, 5, 2, 5])))
        self.t += 1
        done = self.t >= 96
        return self.state, reward, done, False, {}

    def render(self):
        print(f"Step {self.t}, State: {self.state}")

In [7]:
# Instalar librerías necesarias
!pip install gymnasium stable-baselines3[extra] optuna -q

# Imports
import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
from gymnasium import spaces

# Simulación simplificada del BSM1 (mock)
class BSM1Env(gym.Env):
    def __init__(self):
        super(BSM1Env, self).__init__()
        self.action_space = spaces.Box(low=0, high=1, shape=(1,), dtype=np.float32)  # Control de oxígeno
        self.observation_space = spaces.Box(low=0, high=100, shape=(4,), dtype=np.float32)  # DQO, NH4, NO3, SRT
        self.state = np.array([50.0, 10.0, 5.0, 8.0], dtype=np.float32)
        self.t = 0

    def reset(self, seed=None, options=None):
        super().reset(seed=seed)
        self.state = np.array([50.0, 10.0, 5.0, 8.0], dtype=np.float32)
        self.t = 0
        return self.state, {}

    def step(self, action):
        oxigeno = action[0]
        self.state += np.random.normal(0, 0.5, size=self.state.shape) - oxigeno
        reward = -np.sum(np.square(self.state - np.array([30, 5, 2, 5])))  # Penaliza desviación del estado deseado
        self.t += 1
        done = self.t >= 96
        return self.state, reward, done, False, {}

    def render(self):
        print(f"Step {self.t}, State: {self.state}")

# Crear y verificar el entorno
env = BSM1Env()
check_env(env)

# Entrenar el agente PPO
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=5000)

# Guardar el modelo entrenado
model.save("ppo_bsm1_agent")

# Probar el agente entrenado
obs, _ = env.reset()
for i in range(24):
    action, _states = model.predict(obs)
    obs, reward, done, truncated, info = env.step(action)
    env.render()
    if done:
        break

/usr/local/lib/python3.11/dist-packages/stable_baselines3/common/env_checker.py:462: UserWarning: We recommend you to use a symmetric and normalized Box action space (range=[-1, 1]) cf. https://stable-baselines3.readthedocs.io/en/master/guide/rl_tips.html
  warnings.warn(


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
----------------------------------
| rollout/           |           |
|    ep_len_mean     | 96        |
|    ep_rew_mean     | -7.77e+04 |
| time/              |           |
|    fps             | 1339      |
|    iterations      | 1         |
|    time_elapsed    | 1         |
|    total_timesteps | 2048      |
----------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 96           |
|    ep_rew_mean          | -7.71e+04    |
| time/                   |              |
|    fps                  | 921          |
|    iterations           | 2            |
|    time_elapsed         | 4            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0008883655 |
|    clip_fraction        | 0            |
|    clip_range           | 0.2         

In [8]:
from google.colab import files
files.download("ppo_bsm1_agent.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>